# Configurações iniciais:

## Ative o uso da GPU em:
Runtime > Change runtime type > Hardware Accelerator > GPU > Save

In [1]:
!nvidia-smi

Sun Nov  6 16:42:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


## Adiciona o plugin para poder usar o CUDA:

In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-zwtnw64s
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-zwtnw64s


In [4]:
%load_ext nvcc_plugin

directory /content/src already exists
Out bin /content/result.out


# "Hello World" em CUDA:

In [5]:
%%cu
#include <stdio.h>

//#   Atributo	Executa no	 Chamado por	 Nota
//# __global__	Device	     Host	         Precisa ser void
//# __device__	Device	     Device	       Pode retornar qualquer tipo
//# __host__	  Host	       Host	         Opcional

__global__ void hello() {
    printf("Hello World da GPU!\n");
}

int main() {
    hello<<<1, 1>>>(); // <<<blocos, threads por bloco>>>, executa de forma assíncrona!
    printf("Hello World da CPU!\n");
    cudaDeviceSynchronize(); // espera até o kernel finalizar
    return 0;
}

Hello World da CPU!
Hello World da GPU!



# Primeiro exemplo - soma de vetores:

## Versão serial na CPU:

In [6]:
%%cu
#include <chrono>
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define N 100000000

using namespace std::chrono;

void soma_vetores(float *out, float *a, float *b, int n) {
    for (int i = 0; i < n; i++)
        out[i] = a[i] + b[i];
}

int main() {
    float *a, *b, *out; // vetores na memória RAM

    // Aloca os vetores no host (memória RAM):
    a   = (float*) malloc(sizeof(float) * N);
    b   = (float*) malloc(sizeof(float) * N);
    out = (float*) malloc(sizeof(float) * N);

    // Inicializa os vetores:
    for (int i = 0; i < N; i++) {
        a[i] = 1.0f;
        b[i] = 2.0f;
    }

    //############ COMPUTAÇÃO ############
    auto comeco = std::chrono::high_resolution_clock::now();
    soma_vetores(out, a, b, N); // executa a soma
    auto fim = std::chrono::high_resolution_clock::now();

    printf("out[0] = %.3f\n", out[0]);
    double tempo_gasto = duration_cast<milliseconds>(fim - comeco).count();
    printf("Tempo gasto: %.2lfms\n", tempo_gasto);

    // Libera os vetores:
    free(a); 
    free(b);
    free(out);

    return 0;
}

out[0] = 3.000
Tempo gasto: 478.00ms



## Versão GPU:

In [7]:
%%cu
#include <chrono>
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define N 100000000
#define ERRO_MAX 1e-6

using namespace std::chrono;

__global__ void soma_vetores(float *out, float *a, float *b, int n) {
    for (int i = 0; i < n; i++)
        out[i] = a[i] + b[i];
}

int main() {
    float *a, *b, *out; // vetores na memória RAM 
    float *d_a, *d_b, *d_out; // vetores na memória da GPU

    // Aloca os vetores no host (memória RAM):
    a   = (float*) malloc(sizeof(float) * N);
    b   = (float*) malloc(sizeof(float) * N);
    out = (float*) malloc(sizeof(float) * N);

    // Inicializa os vetores:
    for (int i = 0; i < N; i++) {
        a[i] = 1.0f;
        b[i] = 2.0f;
    }

    // Aloca os vetores no device (memória da GPU):
    cudaMalloc((void**) &d_a, sizeof(float) * N);
    cudaMalloc((void**) &d_b, sizeof(float) * N);
    cudaMalloc((void**) &d_out, sizeof(float) * N);

    //###### TRANSFERÊNCIA DE DADOS ######
    // Transfere os dados para a GPU:
    cudaMemcpy(d_a, a, sizeof(float) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(float) * N, cudaMemcpyHostToDevice);

    //############ COMPUTAÇÃO ############
    auto comeco = std::chrono::high_resolution_clock::now();
    soma_vetores<<<1, 1>>>(d_out, d_a, d_b, N); // executa a soma
    cudaDeviceSynchronize(); // espera até o kernel finalizar
    auto fim = std::chrono::high_resolution_clock::now();
    
    //###### TRANSFERÊNCIA DE DADOS ######
    // Transfere os dados de volta da GPU:
    cudaMemcpy(out, d_out, sizeof(float) * N, cudaMemcpyDeviceToHost);

    // Testa a resposta:
    for (int i = 0; i < N; i++)
        assert(fabs(out[i] - a[i] - b[i]) < ERRO_MAX);
    printf("out[0] = %.3f\n", out[0]);
    printf("A soma funcionou!\n");

    double tempo_gasto = duration_cast<milliseconds>(fim - comeco).count();
    printf("Tempo gasto: %.2lfms\n", tempo_gasto);

    // Libera os vetores:
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);
    free(a); 
    free(b);
    free(out);

    return 0;
}

out[0] = 3.000
A soma funcionou!
Tempo gasto: 5454.00ms



## Versão GPU paralela:

In [8]:
%%cu
#include <chrono>
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define N 100000000
#define ERRO_MAX 1e-6

using namespace std::chrono;

__global__ void soma_vetores(float *out, float *a, float *b, int n) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < n)
        out[i] = a[i] + b[i];
}

int main() {
    float *a, *b, *out; // vetores na memória RAM 
    float *d_a, *d_b, *d_out; // vetores na memória da GPU

    // Aloca os vetores no host (memória RAM):
    a   = (float*) malloc(sizeof(float) * N);
    b   = (float*) malloc(sizeof(float) * N);
    out = (float*) malloc(sizeof(float) * N);

    // Inicializa os vetores:
    for (int i = 0; i < N; i++) {
        a[i] = 1.0f;
        b[i] = 2.0f;
    }

    // Aloca os vetores no device (memória da GPU):
    cudaMalloc((void**) &d_a, sizeof(float) * N);
    cudaMalloc((void**) &d_b, sizeof(float) * N);
    cudaMalloc((void**) &d_out, sizeof(float) * N);

    //###### TRANSFERÊNCIA DE DADOS ######
    // Transfere os dados para a GPU:
    cudaMemcpy(d_a, a, sizeof(float) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(float) * N, cudaMemcpyHostToDevice);

    //############ COMPUTAÇÃO ############
    auto comeco = std::chrono::high_resolution_clock::now();
    int threads = 1024;
    int blocos = ((N + threads) / threads);
    soma_vetores<<<blocos, threads>>>(d_out, d_a, d_b, N); // executa a soma
    cudaDeviceSynchronize(); // espera até o kernel finalizar
    auto fim = std::chrono::high_resolution_clock::now();

    //###### TRANSFERÊNCIA DE DADOS ######
    // Transfere os dados de volta da GPU:
    cudaMemcpy(out, d_out, sizeof(float) * N, cudaMemcpyDeviceToHost);

    // Testa a resposta:
    for(int i = 0; i < N; i++)
        assert(fabs(out[i] - a[i] - b[i]) < ERRO_MAX);
    printf("out[0] = %.3f\n", out[0]);
    printf("A soma funcionou!\n");

    double tempo_gasto = duration_cast<milliseconds>(fim - comeco).count();
    printf("Tempo gasto: %.2lfms\n", tempo_gasto);

    // Libera os vetores:
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);
    free(a); 
    free(b);
    free(out);

    return 0;
}

out[0] = 3.000
A soma funcionou!
Tempo gasto: 4.00ms



# Segundo exemplo - convolução 2D

Configura o acesso a arquivos no Drive:

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Versão serial na CPU:

In [10]:
%%cu
#include <chrono>
#include <cmath>
#include <cstdio>
#include <cstdlib>
#include <sys/time.h>

#define LARGURA_MASCARA 3// número ímpar
#define RAIO 1           // (LARGURA_MASCARA - 1) / 2

#define COMENTARIO "Imagem_Borrada"
#define RGB_MAX 255

using namespace std::chrono;

typedef struct {
    unsigned char vermelho, verde, azul;
} PPMPixel;

typedef struct {
    int x, y;
    PPMPixel *imagem;
} PPMImagem;

PPMImagem *lePPM(const char *nome_arquivo, char pula_pixeis) {
    char buff[16];
    PPMImagem *img;
    FILE *fp;
    int c, rgb_max;
    fp = fopen(nome_arquivo, "rb");
    if (!fp) {
        fprintf(stdout, "Incapaz de abrir o arquivo '%s'\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (!fgets(buff, sizeof(buff), fp)) {// lê o formato da imagem
        perror(nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (buff[0] != 'P' || buff[1] != '6') {// valida o formato da imagem
        fprintf(stdout, "Formato de imagem inválido (precisa ser 'P6')\n");
        exit(EXIT_FAILURE);
    }

    c = getc(fp);
    while (c == '#') {// ignora os comentários
        while (getc(fp) != '\n');
        c = getc(fp);
    }

    img = (PPMImagem *) malloc(sizeof(PPMImagem));
    ungetc(c, fp);
    if (fscanf(fp, "%d %d", &img->x, &img->y) != 2) {
        fprintf(stdout, "Tamahno inválido (erro ao carregar '%s')\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (fscanf(fp, "%d", &rgb_max) != 1) {
        fprintf(stdout, "Valor máximo por componente RGB inválido (erro ao carregar '%s')\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (rgb_max != RGB_MAX) {
        fprintf(stdout, "'%s' não possui componentes de 8-bits\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    img->imagem = (PPMPixel *) malloc(img->x * img->y * sizeof(PPMPixel));
    if (!pula_pixeis) {
        while (fgetc(fp) != '\n');

        if (fread(img->imagem, 3 * img->x, img->y, fp) != img->y) {// lê os pixeis
            fprintf(stdout, "Erro ao carregar imagem '%s'\n", nome_arquivo);
            exit(EXIT_FAILURE);
        }
    }

    fclose(fp);
    return img;
}

void escrevePPM(PPMImagem *img, const char *nome_arquivo_out) {
    FILE *fp = fopen(nome_arquivo_out, "wb");
    fprintf(fp, "P6\n");
    fprintf(fp, "# %s\n", COMENTARIO);
    fprintf(fp, "%d %d\n", img->x, img->y);
    fprintf(fp, "%d\n", RGB_MAX);

    fwrite(img->imagem, 3 * img->x, img->y, fp);
    fclose(fp);
}

unsigned char *aloca_vet(int tam) {
    return (unsigned char *) malloc(tam * sizeof(unsigned char));
}

void borra_imagem(
        unsigned char *vermelho,
        unsigned char *verde,
        unsigned char *azul,
        int x, int y) {
    int tam = x * y;
    int *vermelho_aux = (int *) malloc(tam * sizeof(int)),
        *verde_aux = (int *) malloc(tam * sizeof(int)),
        *azul_aux = (int *) malloc(tam * sizeof(int));
    unsigned char *vermelho_out = aloca_vet(tam),
        *verde_out = aloca_vet(tam),
        *azul_out = aloca_vet(tam);

    for (int i = 0; i < y; i++)
        for (int j = 0; j < x; j++) {
            vermelho_aux[i * x + j] = verde_aux[i * x + j] = azul_aux[i * x + j] = 0;
            for (int i_offset = i - RAIO; i_offset <= i + RAIO; i_offset++)
                for (int j_offset = j - RAIO; j_offset <= j + RAIO; j_offset++) {
                    if (0 <= i && i < y && 0 <= j && j < x) {
                      vermelho_aux[i * x + j] += vermelho[i_offset * x + j_offset];
                      verde_aux[i * x + j] += verde[i_offset * x + j_offset];
                      azul_aux[i * x + j] += azul[i_offset * x + j_offset];
                    }
                }
            vermelho_out[i * x + j] =  vermelho_aux[i * x + j] / (LARGURA_MASCARA * LARGURA_MASCARA);
            verde_out[i * x + j] =  verde_aux[i * x + j] / (LARGURA_MASCARA * LARGURA_MASCARA);
            azul_out[i * x + j] =  azul_aux[i * x + j] / (LARGURA_MASCARA * LARGURA_MASCARA);
        }

    memcpy(vermelho, vermelho_out, tam * sizeof(unsigned char));
    memcpy(verde, verde_out, tam * sizeof(unsigned char));
    memcpy(azul, azul_out, tam * sizeof(unsigned char));

    free(vermelho_aux);
    free(verde_aux);
    free(azul_aux);
    free(vermelho_out);
    free(verde_out);
    free(azul_out);
}

void canais_da_imagem(
        const PPMImagem *img,
        unsigned char **vermelho,
        unsigned char **verde,
        unsigned char **azul, int tam) {
    *vermelho = aloca_vet(tam);
    *verde = aloca_vet(tam);
    *azul = aloca_vet(tam);

    for (int i = 0; i < tam; i++) {
        (*vermelho)[i] = img->imagem[i].vermelho;
        (*verde)[i] = img->imagem[i].verde;
        (*azul)[i] = img->imagem[i].azul;
    }
}

void imagem_dos_canais(
        const unsigned char *vermelho,
        const unsigned char *verde,
        const unsigned char *azul,
        PPMImagem *img, int tam) {
    for (int i = 0; i < tam; i++) {
        img->imagem[i].vermelho = vermelho[i];
        img->imagem[i].verde = verde[i];
        img->imagem[i].azul = azul[i];
    }
}

int main() {
    unsigned char *vermelho, *verde, *azul;// canais da imagem de entrada
    const char nome_arquivo[] = "/content/drive/MyDrive/secomp/4.ppm";
    const char nome_arquivo_out[] = "/content/drive/MyDrive/secomp/4_out.ppm";

    // Lê a imagem de entrada:
    PPMImagem *img = lePPM(nome_arquivo, 0);
    PPMImagem *img_output = lePPM(nome_arquivo, 1);
    int tam = img->x * img->y;

    canais_da_imagem(img, &vermelho, &verde, &azul, tam);
    auto comeco = std::chrono::high_resolution_clock::now();

    borra_imagem(vermelho, verde, azul, img->x, img->y);

    auto fim = std::chrono::high_resolution_clock::now();
    imagem_dos_canais(vermelho, verde, azul, img_output, tam);

    escrevePPM(img_output, nome_arquivo_out);
    double tempo_gasto = duration_cast<milliseconds>(fim - comeco).count();
    printf("Tempo gasto: %.2lfms\n", tempo_gasto);

    // Libera os dados alocados:
    free(img->imagem);
    free(img);
    free(img_output->imagem);
    free(img_output);
    free(vermelho);
    free(verde);
    free(azul);

    return 0;
}

Tempo gasto: 3353.00ms



## Versão GPU básica:

In [11]:
%%cu
#include <chrono>
#include <cmath>
#include <cstdio>
#include <cstdlib>

#define LARGURA_MASCARA 31// número ímpar
#define RAIO 15           // (LARGURA_MASCARA - 1) / 2

#define COMENTARIO "Imagem_Borrada"
#define RGB_MAX 255

#define LADO_BLOCO 32// os blocos 2D terão LADO_BLOCOxLADO_BLOCO threads

using namespace std::chrono;

void checacuda(const cudaError_t erro, const char *nome_arquivo, const int linha) {
    if (erro != cudaSuccess) {
        fprintf(stdout, "Erro: %s:%d: %s: %s\n", nome_arquivo, linha,
                cudaGetErrorName(erro), cudaGetErrorString(erro));
        exit(EXIT_FAILURE);
    }
}

#define CHECA_CUDA(cmd) checacuda(cmd, __FILE__, __LINE__)

typedef struct {
    unsigned char vermelho, verde, azul;
} PPMPixel;

typedef struct {
    int x, y;
    PPMPixel *imagem;
} PPMImagem;

PPMImagem *lePPM(const char *nome_arquivo, char pula_pixeis) {
    char buff[16];
    PPMImagem *img;
    FILE *fp;
    int c, rgb_max;
    fp = fopen(nome_arquivo, "rb");
    if (!fp) {
        fprintf(stdout, "Incapaz de abrir o arquivo '%s'\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (!fgets(buff, sizeof(buff), fp)) {// lê o formato da imagem
        perror(nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (buff[0] != 'P' || buff[1] != '6') {// valida o formato da imagem
        fprintf(stdout, "Formato de imagem inválido (precisa ser 'P6')\n");
        exit(EXIT_FAILURE);
    }

    c = getc(fp);
    while (c == '#') {// ignora os comentários
        while (getc(fp) != '\n');
        c = getc(fp);
    }

    img = (PPMImagem *) malloc(sizeof(PPMImagem));
    ungetc(c, fp);
    if (fscanf(fp, "%d %d", &img->x, &img->y) != 2) {
        fprintf(stdout, "Tamahno inválido (erro ao carregar '%s')\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (fscanf(fp, "%d", &rgb_max) != 1) {
        fprintf(stdout, "Valor máximo por componente RGB inválido (erro ao carregar '%s')\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (rgb_max != RGB_MAX) {
        fprintf(stdout, "'%s' não possui componentes de 8-bits\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    img->imagem = (PPMPixel *) malloc(img->x * img->y * sizeof(PPMPixel));
    if (!pula_pixeis) {
        while (fgetc(fp) != '\n');

        if (fread(img->imagem, 3 * img->x, img->y, fp) != img->y) {// lê os pixeis
            fprintf(stdout, "Erro ao carregar imagem '%s'\n", nome_arquivo);
            exit(EXIT_FAILURE);
        }
    }

    fclose(fp);
    return img;
}

void escrevePPM(PPMImagem *img, const const char *nome_arquivo_out) {
    FILE *fp = fopen(nome_arquivo_out, "wb");
    fprintf(fp, "P6\n");
    fprintf(fp, "# %s\n", COMENTARIO);
    fprintf(fp, "%d %d\n", img->x, img->y);
    fprintf(fp, "%d\n", RGB_MAX);

    fwrite(img->imagem, 3 * img->x, img->y, fp);
    fclose(fp);
}

__global__ void borra_X(unsigned char *canal, int *canal_out, int N, int M) {
    // Coordenadas globais da thread:
    int linha = LADO_BLOCO * blockIdx.y + threadIdx.y;
    int coluna = LADO_BLOCO * blockIdx.x + threadIdx.x;

    if (linha >= N || coluna >= M)
        return;// threads fora da imagem

    int gindice = linha * M + coluna;// indice global da thread dentro do canal
    // Aplica o stencil:
    int result = 0;
    for (int offset = -RAIO; offset <= RAIO; offset++)
        if (0 <= coluna + offset && coluna + offset < M)// valores fora da imagem são considerados como zero
            result += canal[gindice + offset];

    // Salva o resultado final na memória global:
    canal_out[gindice] = result;
}

__global__ void borra_Y(int *canal, unsigned char *canal_out, int N, int M) {
    // Coordenadas globais da thread:
    int linha = LADO_BLOCO * blockIdx.y + threadIdx.y;
    int coluna = LADO_BLOCO * blockIdx.x + threadIdx.x;

    if (linha >= N || coluna >= M)
        return;// threads fora da imagem

    int gindice = linha * M + coluna;// indice global da thread dentro do canal
    // Aplica o stencil:
    int result = 0;
    for (int offset = -RAIO; offset <= RAIO; offset++)
        if (0 <= linha + offset && linha + offset < N)// valores fora da imagem são considerados como zero
            result += canal[gindice + offset * M];

    // Salva o resultado final na memória global calculando a divisão faltante:
    canal_out[gindice] = result / (LARGURA_MASCARA * LARGURA_MASCARA);
}

void borra_imagem(
        unsigned char *vermelho,
        unsigned char *verde,
        unsigned char *azul,
        int x, int y) {
    // As saídas intermediárias serão armazenadas em vetores de inteiros para evitar overflow:
    size_t tam_char = x * y * sizeof(unsigned char);
    size_t tam_int = x * y * sizeof(int);
    unsigned char *d_vermelho, *d_verde, *d_azul;
    int *d_vermelho_aux, *d_verde_aux, *d_azul_aux;

    // Aloca os canais na GPU:
    CHECA_CUDA(cudaMalloc(&d_vermelho, tam_char));
    CHECA_CUDA(cudaMalloc(&d_verde, tam_char));
    CHECA_CUDA(cudaMalloc(&d_azul, tam_char));
    CHECA_CUDA(cudaMalloc(&d_vermelho_aux, tam_int));
    CHECA_CUDA(cudaMalloc(&d_verde_aux, tam_int));
    CHECA_CUDA(cudaMalloc(&d_azul_aux, tam_int));

    dim3 dimGrade(ceil((float) x / LADO_BLOCO), ceil((float) y / LADO_BLOCO), 1);
    dim3 dimBloco(LADO_BLOCO, LADO_BLOCO, 1);

    // borra_X:
    CHECA_CUDA(cudaMemcpy(d_vermelho, vermelho, tam_char, cudaMemcpyHostToDevice));
    borra_X<<<dimGrade, dimBloco>>>(d_vermelho, d_vermelho_aux, y, x);

    CHECA_CUDA(cudaMemcpy(d_verde, verde, tam_char, cudaMemcpyHostToDevice));
    borra_X<<<dimGrade, dimBloco>>>(d_verde, d_verde_aux, y, x);

    CHECA_CUDA(cudaMemcpy(d_azul, azul, tam_char, cudaMemcpyHostToDevice));
    borra_X<<<dimGrade, dimBloco>>>(d_azul, d_azul_aux, y, x);

    // borra_Y:
    borra_Y<<<dimGrade, dimBloco>>>(d_vermelho_aux, d_vermelho, y, x);
    CHECA_CUDA(cudaMemcpy(vermelho, d_vermelho, tam_char, cudaMemcpyDeviceToHost));

    borra_Y<<<dimGrade, dimBloco>>>(d_verde_aux, d_verde, y, x);
    CHECA_CUDA(cudaMemcpy(verde, d_verde, tam_char, cudaMemcpyDeviceToHost));

    borra_Y<<<dimGrade, dimBloco>>>(d_azul_aux, d_azul, y, x);
    CHECA_CUDA(cudaMemcpy(azul, d_azul, tam_char, cudaMemcpyDeviceToHost));

    CHECA_CUDA(cudaFree(d_vermelho));
    CHECA_CUDA(cudaFree(d_verde));
    CHECA_CUDA(cudaFree(d_azul));
    CHECA_CUDA(cudaFree(d_vermelho_aux));
    CHECA_CUDA(cudaFree(d_verde_aux));
    CHECA_CUDA(cudaFree(d_azul_aux));
}

unsigned char *aloca_vet(int tam) {
    return (unsigned char *) malloc(tam * sizeof(unsigned char));
}

void canais_da_imagem(
        const PPMImagem *img,
        unsigned char **vermelho,
        unsigned char **verde,
        unsigned char **azul, int tam) {
    *vermelho = aloca_vet(tam);
    *verde = aloca_vet(tam);
    *azul = aloca_vet(tam);

    for (int i = 0; i < tam; i++) {
        (*vermelho)[i] = img->imagem[i].vermelho;
        (*verde)[i] = img->imagem[i].verde;
        (*azul)[i] = img->imagem[i].azul;
    }
}

void imagem_dos_canais(
        const unsigned char *vermelho,
        const unsigned char *verde,
        const unsigned char *azul,
        PPMImagem *img, int tam) {
    for (int i = 0; i < tam; i++) {
        img->imagem[i].vermelho = vermelho[i];
        img->imagem[i].verde = verde[i];
        img->imagem[i].azul = azul[i];
    }
}

int main() {
    unsigned char *vermelho, *verde, *azul;// canais da imagem de entrada
    const char nome_arquivo[] = "/content/drive/MyDrive/secomp/4.ppm";
    const char nome_arquivo_out[] = "/content/drive/MyDrive/secomp/4_out.ppm";

    // Lê a imagem de entrada:
    PPMImagem *img = lePPM(nome_arquivo, 0);
    PPMImagem *img_output = lePPM(nome_arquivo, 1);
    int tam = img->x * img->y;

    canais_da_imagem(img, &vermelho, &verde, &azul, tam);
    auto comeco = std::chrono::high_resolution_clock::now();

    borra_imagem(vermelho, verde, azul, img->x, img->y);
    CHECA_CUDA(cudaDeviceSynchronize());// espera até os kerneis finalizarem

    auto fim = std::chrono::high_resolution_clock::now();
    imagem_dos_canais(vermelho, verde, azul, img_output, tam);

    escrevePPM(img_output, nome_arquivo_out);
    double tempo_gasto = duration_cast<milliseconds>(fim - comeco).count();
    printf("Tempo gasto: %.2lfms\n", tempo_gasto);

    // Libera os dados alocados:
    free(img->imagem);
    free(img);
    free(img_output->imagem);
    free(img_output);
    free(vermelho);
    free(verde);
    free(azul);

    return 0;
}

Tempo gasto: 206.00ms



## Versão GPU com multiplas streams:

In [12]:
%%cu
#include <chrono>
#include <cmath>
#include <cstdio>
#include <cstdlib>
#include <ctime>

#define LARGURA_MASCARA 31// número ímpar
#define RAIO 15           // (LARGURA_MASCARA - 1) / 2

#define COMENTARIO "Imagem_Borrada"
#define RGB_MAX 255

#define LADO_BLOCO 32// os blocos 2D terão LADO_BLOCOxLADO_BLOCO threads

using namespace std::chrono;

void checacuda(const cudaError_t erro, const char *nome_arquivo, const int linha) {
    if (erro != cudaSuccess) {
        fprintf(stdout, "Erro: %s:%d: %s: %s\n", nome_arquivo, linha,
                cudaGetErrorName(erro), cudaGetErrorString(erro));
        exit(EXIT_FAILURE);
    }
}

#define CHECA_CUDA(cmd) checacuda(cmd, __FILE__, __LINE__)

typedef struct {
    unsigned char vermelho, verde, azul;
} PPMPixel;

typedef struct {
    int x, y;
    PPMPixel *imagem;
} PPMImagem;

PPMImagem *lePPM(const char *nome_arquivo, char pula_pixeis) {
    char buff[16];
    PPMImagem *img;
    FILE *fp;
    int c, rgb_max;
    fp = fopen(nome_arquivo, "rb");
    if (!fp) {
        fprintf(stdout, "Incapaz de abrir o arquivo '%s'\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (!fgets(buff, sizeof(buff), fp)) {// lê o formato da imagem
        perror(nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (buff[0] != 'P' || buff[1] != '6') {// valida o formato da imagem
        fprintf(stdout, "Formato de imagem inválido (precisa ser 'P6')\n");
        exit(EXIT_FAILURE);
    }

    c = getc(fp);
    while (c == '#') {// ignora os comentários
        while (getc(fp) != '\n');
        c = getc(fp);
    }

    img = (PPMImagem *) malloc(sizeof(PPMImagem));
    ungetc(c, fp);
    if (fscanf(fp, "%d %d", &img->x, &img->y) != 2) {
        fprintf(stdout, "Tamahno inválido (erro ao carregar '%s')\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (fscanf(fp, "%d", &rgb_max) != 1) {
        fprintf(stdout, "Valor máximo por componente RGB inválido (erro ao carregar '%s')\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (rgb_max != RGB_MAX) {
        fprintf(stdout, "'%s' não possui componentes de 8-bits\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    img->imagem = (PPMPixel *) malloc(img->x * img->y * sizeof(PPMPixel));
    if (!pula_pixeis) {
        while (fgetc(fp) != '\n');

        if (fread(img->imagem, 3 * img->x, img->y, fp) != img->y) {// lê os pixeis
            fprintf(stdout, "Erro ao carregar imagem '%s'\n", nome_arquivo);
            exit(EXIT_FAILURE);
        }
    }

    fclose(fp);
    return img;
}

void escrevePPM(PPMImagem *img, const const char *nome_arquivo_out) {
    FILE *fp = fopen(nome_arquivo_out, "wb");
    fprintf(fp, "P6\n");
    fprintf(fp, "# %s\n", COMENTARIO);
    fprintf(fp, "%d %d\n", img->x, img->y);
    fprintf(fp, "%d\n", RGB_MAX);

    fwrite(img->imagem, 3 * img->x, img->y, fp);
    fclose(fp);
}

__global__ void borra_X(unsigned char *canal, int *canal_out, int N, int M) {
    // Coordenadas globais da thread:
    int linha = LADO_BLOCO * blockIdx.y + threadIdx.y;
    int coluna = LADO_BLOCO * blockIdx.x + threadIdx.x;

    if (linha >= N || coluna >= M)
        return;// threads fora da imagem

    int gindice = linha * M + coluna;// indice global da thread dentro do canal
    // Aplica o stencil:
    int result = 0;
    for (int offset = -RAIO; offset <= RAIO; offset++)
        if (0 <= coluna + offset && coluna + offset < M)// valores fora da imagem são considerados como zero
            result += canal[gindice + offset];

    // Salva o resultado final na memória global:
    canal_out[gindice] = result;
}

__global__ void borra_Y(int *canal, unsigned char *canal_out, int N, int M) {
    // Coordenadas globais da thread:
    int linha = LADO_BLOCO * blockIdx.y + threadIdx.y;
    int coluna = LADO_BLOCO * blockIdx.x + threadIdx.x;

    if (linha >= N || coluna >= M)
        return;// threads fora da imagem

    int gindice = linha * M + coluna;// indice global da thread dentro do canal
    // Aplica o stencil:
    int result = 0;
    for (int offset = -RAIO; offset <= RAIO; offset++)
        if (0 <= linha + offset && linha + offset < N)// valores fora da imagem são considerados como zero
            result += canal[gindice + offset * M];

    // Salva o resultado final na memória global calculando a divisão faltante:
    canal_out[gindice] = result / (LARGURA_MASCARA * LARGURA_MASCARA);
}

void borra_imagem(
        unsigned char *vermelho,
        unsigned char *verde,
        unsigned char *azul,
        int x, int y) {
    // As saídas intermediárias serão armazenadas em vetores de inteiros para evitar overflow:
    size_t tam_char = x * y * sizeof(unsigned char);
    size_t tam_int = x * y * sizeof(int);
    unsigned char *d_vermelho, *d_verde, *d_azul;
    int *d_vermelho_aux, *d_verde_aux, *d_azul_aux;

    // Cria uma Stream para cada canal da imagem:
    cudaStream_t stream_vermelho, stream_verde, stream_azul;
    CHECA_CUDA(cudaStreamCreate(&stream_vermelho));
    CHECA_CUDA(cudaStreamCreate(&stream_verde));
    CHECA_CUDA(cudaStreamCreate(&stream_azul));

    // Aloca os canais na GPU:
    CHECA_CUDA(cudaMalloc(&d_vermelho, tam_char));
    CHECA_CUDA(cudaMalloc(&d_verde, tam_char));
    CHECA_CUDA(cudaMalloc(&d_azul, tam_char));
    CHECA_CUDA(cudaMalloc(&d_vermelho_aux, tam_int));
    CHECA_CUDA(cudaMalloc(&d_verde_aux, tam_int));
    CHECA_CUDA(cudaMalloc(&d_azul_aux, tam_int));

    dim3 dimGrade(ceil((float) x / LADO_BLOCO), ceil((float) y / LADO_BLOCO), 1);
    dim3 dimBloco(LADO_BLOCO, LADO_BLOCO, 1);

    // borra_X:
    CHECA_CUDA(cudaMemcpyAsync(d_vermelho, vermelho, tam_char, cudaMemcpyHostToDevice, stream_vermelho));
    borra_X<<<dimGrade, dimBloco, 0, stream_vermelho>>>(d_vermelho, d_vermelho_aux, y, x);

    CHECA_CUDA(cudaMemcpyAsync(d_verde, verde, tam_char, cudaMemcpyHostToDevice, stream_verde));
    borra_X<<<dimGrade, dimBloco, 0, stream_verde>>>(d_verde, d_verde_aux, y, x);

    CHECA_CUDA(cudaMemcpyAsync(d_azul, azul, tam_char, cudaMemcpyHostToDevice, stream_azul));
    borra_X<<<dimGrade, dimBloco, 0, stream_azul>>>(d_azul, d_azul_aux, y, x);

    // borra_Y:
    borra_Y<<<dimGrade, dimBloco, 0, stream_vermelho>>>(d_vermelho_aux, d_vermelho, y, x);
    CHECA_CUDA(cudaMemcpyAsync(vermelho, d_vermelho, tam_char, cudaMemcpyDeviceToHost, stream_vermelho));

    borra_Y<<<dimGrade, dimBloco, 0, stream_verde>>>(d_verde_aux, d_verde, y, x);
    CHECA_CUDA(cudaMemcpyAsync(verde, d_verde, tam_char, cudaMemcpyDeviceToHost, stream_verde));

    borra_Y<<<dimGrade, dimBloco, 0, stream_azul>>>(d_azul_aux, d_azul, y, x);
    CHECA_CUDA(cudaMemcpyAsync(azul, d_azul, tam_char, cudaMemcpyDeviceToHost, stream_azul));

    CHECA_CUDA(cudaStreamDestroy(stream_vermelho));
    CHECA_CUDA(cudaStreamDestroy(stream_verde));
    CHECA_CUDA(cudaStreamDestroy(stream_azul));

    CHECA_CUDA(cudaFree(d_vermelho));
    CHECA_CUDA(cudaFree(d_verde));
    CHECA_CUDA(cudaFree(d_azul));
    CHECA_CUDA(cudaFree(d_vermelho_aux));
    CHECA_CUDA(cudaFree(d_verde_aux));
    CHECA_CUDA(cudaFree(d_azul_aux));
}

unsigned char *aloca_vet(int tam) {
    return (unsigned char *) malloc(tam * sizeof(unsigned char));
}

void canais_da_imagem(
        const PPMImagem *img,
        unsigned char **vermelho,
        unsigned char **verde,
        unsigned char **azul, int tam) {
    *vermelho = aloca_vet(tam);
    *verde = aloca_vet(tam);
    *azul = aloca_vet(tam);

    for (int i = 0; i < tam; i++) {
        (*vermelho)[i] = img->imagem[i].vermelho;
        (*verde)[i] = img->imagem[i].verde;
        (*azul)[i] = img->imagem[i].azul;
    }
}

void imagem_dos_canais(
        const unsigned char *vermelho,
        const unsigned char *verde,
        const unsigned char *azul,
        PPMImagem *img, int tam) {
    for (int i = 0; i < tam; i++) {
        img->imagem[i].vermelho = vermelho[i];
        img->imagem[i].verde = verde[i];
        img->imagem[i].azul = azul[i];
    }
}

int main() {
    unsigned char *vermelho, *verde, *azul;// canais da imagem de entrada
    const char nome_arquivo[] = "/content/drive/MyDrive/secomp/4.ppm";
    const char nome_arquivo_out[] = "/content/drive/MyDrive/secomp/4_out.ppm";

    // Lê a imagem de entrada:
    PPMImagem *img = lePPM(nome_arquivo, 0);
    PPMImagem *img_output = lePPM(nome_arquivo, 1);
    int tam = img->x * img->y;

    canais_da_imagem(img, &vermelho, &verde, &azul, tam);
    auto comeco = std::chrono::high_resolution_clock::now();

    borra_imagem(vermelho, verde, azul, img->x, img->y);
    CHECA_CUDA(cudaDeviceSynchronize());// espera até os kerneis finalizarem

    auto fim = std::chrono::high_resolution_clock::now();
    imagem_dos_canais(vermelho, verde, azul, img_output, tam);

    escrevePPM(img_output, nome_arquivo_out);
    double tempo_gasto = duration_cast<milliseconds>(fim - comeco).count();
    printf("Tempo gasto: %.2lfms\n", tempo_gasto);

    // Libera os dados alocados:
    free(img->imagem);
    free(img);
    free(img_output->imagem);
    free(img_output);
    free(vermelho);
    free(verde);
    free(azul);

    return 0;
}

Tempo gasto: 185.00ms



## Versão GPU final com shared memory:

In [13]:
%%cu
#include <chrono>
#include <cmath>
#include <cstdio>
#include <cstdlib>
#include <ctime>

#define LARGURA_MASCARA 31// número ímpar
#define RAIO 15           // (LARGURA_MASCARA - 1) / 2

#define COMENTARIO "Imagem_Borrada"
#define RGB_MAX 255

#define LADO_BLOCO 32// os blocos 2D terão LADO_BLOCOxLADO_BLOCO threads

using namespace std::chrono;

void checacuda(const cudaError_t erro, const char *nome_arquivo, const int linha) {
    if (erro != cudaSuccess) {
        fprintf(stdout, "Erro: %s:%d: %s: %s\n", nome_arquivo, linha,
                cudaGetErrorName(erro), cudaGetErrorString(erro));
        exit(EXIT_FAILURE);
    }
}

#define CHECA_CUDA(cmd) checacuda(cmd, __FILE__, __LINE__)

typedef struct {
    unsigned char vermelho, verde, azul;
} PPMPixel;

typedef struct {
    int x, y;
    PPMPixel *imagem;
} PPMImagem;

PPMImagem *lePPM(const char *nome_arquivo, char pula_pixeis) {
    char buff[16];
    PPMImagem *img;
    FILE *fp;
    int c, rgb_max;
    fp = fopen(nome_arquivo, "rb");
    if (!fp) {
        fprintf(stdout, "Incapaz de abrir o arquivo '%s'\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (!fgets(buff, sizeof(buff), fp)) {// lê o formato da imagem
        perror(nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (buff[0] != 'P' || buff[1] != '6') {// valida o formato da imagem
        fprintf(stdout, "Formato de imagem inválido (precisa ser 'P6')\n");
        exit(EXIT_FAILURE);
    }

    c = getc(fp);
    while (c == '#') {// ignora os comentários
        while (getc(fp) != '\n');
        c = getc(fp);
    }

    img = (PPMImagem *) malloc(sizeof(PPMImagem));
    ungetc(c, fp);
    if (fscanf(fp, "%d %d", &img->x, &img->y) != 2) {
        fprintf(stdout, "Tamahno inválido (erro ao carregar '%s')\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (fscanf(fp, "%d", &rgb_max) != 1) {
        fprintf(stdout, "Valor máximo por componente RGB inválido (erro ao carregar '%s')\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    if (rgb_max != RGB_MAX) {
        fprintf(stdout, "'%s' não possui componentes de 8-bits\n", nome_arquivo);
        exit(EXIT_FAILURE);
    }

    img->imagem = (PPMPixel *) malloc(img->x * img->y * sizeof(PPMPixel));
    if (!pula_pixeis) {
        while (fgetc(fp) != '\n');

        if (fread(img->imagem, 3 * img->x, img->y, fp) != img->y) {// lê os pixeis
            fprintf(stdout, "Erro ao carregar imagem '%s'\n", nome_arquivo);
            exit(EXIT_FAILURE);
        }
    }

    fclose(fp);
    return img;
}

void escrevePPM(PPMImagem *img, const const char *nome_arquivo_out) {
    FILE *fp = fopen(nome_arquivo_out, "wb");
    fprintf(fp, "P6\n");
    fprintf(fp, "# %s\n", COMENTARIO);
    fprintf(fp, "%d %d\n", img->x, img->y);
    fprintf(fp, "%d\n", RGB_MAX);

    fwrite(img->imagem, 3 * img->x, img->y, fp);
    fclose(fp);
}

__global__ void borra_X(unsigned char *canal, int *canal_out, int N, int M) {
    int llinha = threadIdx.y;// linha da thread localmente dentro do bloco
    int lcoluna = threadIdx.x + RAIO;// coluna da thread dentro da matriz compartilhada
    // Coordenadas globais da thread:
    int linha = LADO_BLOCO * blockIdx.y + threadIdx.y;
    int coluna = LADO_BLOCO * blockIdx.x + threadIdx.x;

    __shared__ unsigned char temp[LADO_BLOCO][LARGURA_MASCARA - 1 + LADO_BLOCO];
    temp[llinha][lcoluna] = 0;// inicializa a memória compartilhada

    if (linha >= N || coluna >= M)
        return;// threads fora da imagem

    int gindice = linha * M + coluna;// indice global da thread dentro do canal
    // Cópia otimizada de dados para a memória compartilhada (cada thread copia até dois valores):
    temp[llinha][lcoluna] = canal[gindice];//copia o dado correspondente a essa thread
    if (threadIdx.x < RAIO) {// essa thread vai copiar um dado da borda esquerda
        if (coluna >= RAIO)
            temp[llinha][lcoluna - RAIO] = canal[gindice - RAIO];
        else
            temp[llinha][lcoluna - RAIO] = 0;// se a borda sai da imagem coloca zero
    } else if (threadIdx.x >= blockDim.x - RAIO) {// essa thread vai copiar um dado da borda direita
        if (coluna < M - RAIO)
            temp[llinha][lcoluna + RAIO] = canal[gindice + RAIO];
        else
            temp[llinha][lcoluna + RAIO] = 0;// se a borda sai da imagem coloca zero
    }

    __syncthreads();// espera a memória compartilhada ser completamente populada

    // Aplica o stencil:
    int result = 0;
    for (int offset = -RAIO; offset <= RAIO; offset++)
        result += temp[llinha][lcoluna + offset];

    // Salva o resultado final na memória global:
    canal_out[gindice] = result;
}

__global__ void borra_Y(int *canal, unsigned char *canal_out, int N, int M) {
    int llinha = threadIdx.y + RAIO;// linha da thread dentro da matriz compartilhada (transposta)
    int lcol = threadIdx.x;// coluna da thread localmente dentro do bloco
    // Coordenadas globais da thread:
    int linha = LADO_BLOCO * blockIdx.y + threadIdx.y;
    int coluna = LADO_BLOCO * blockIdx.x + threadIdx.x;

    // A matriz compartilhada está transposta:
    __shared__ int temp[LADO_BLOCO][LARGURA_MASCARA - 1 + LADO_BLOCO];
    temp[lcol][llinha] = 0;// inicializa a memória compartilhada

    if (linha >= N || coluna >= M)
        return;// threads fora da imagem

    int gindice = linha * M + coluna;// indice global da thread dentro do canal
    // Cópia otimizada de dados para a memória compartilhada (cada thread copia até dois valores):
    temp[lcol][llinha] = canal[gindice];//copia o dado correspondente a essa thread
    if (threadIdx.y < RAIO) {// essa thread vai copiar um dado da borda superior
        if (linha >= RAIO)
            temp[lcol][llinha - RAIO] = canal[gindice - RAIO * M];
        else
            temp[lcol][llinha - RAIO] = 0;// se a borda sai da imagem coloca zero
    } else if (threadIdx.y >= blockDim.y - RAIO) {// essa thread vai copiar um dado da borda inferior
        if (linha < N - RAIO)
            temp[lcol][llinha + RAIO] = canal[gindice + RAIO * M];
        else
            temp[lcol][llinha + RAIO] = 0;// se a borda sai da imagem coloca zero
    }

    __syncthreads();// espera a memória compartilhada ser completamente populada

    // Aplica o stencil:
    int result = 0;
    for (int offset = -RAIO; offset <= RAIO; offset++)
        result += temp[lcol][llinha + offset];

    // Salva o resultado final na memória global calculando a divisão faltante:
    canal_out[gindice] = result / (LARGURA_MASCARA * LARGURA_MASCARA);
}

void borra_imagem(
        unsigned char *vermelho,
        unsigned char *verde,
        unsigned char *azul,
        int x, int y) {
    // As saídas intermediárias serão armazenadas em vetores de inteiros para evitar overflow:
    size_t tam_char = x * y * sizeof(unsigned char);
    size_t tam_int = x * y * sizeof(int);
    unsigned char *d_vermelho, *d_verde, *d_azul;
    int *d_vermelho_aux, *d_verde_aux, *d_azul_aux;

    // Cria uma Stream para cada canal da imagem:
    cudaStream_t stream_vermelho, stream_verde, stream_azul;
    CHECA_CUDA(cudaStreamCreate(&stream_vermelho));
    CHECA_CUDA(cudaStreamCreate(&stream_verde));
    CHECA_CUDA(cudaStreamCreate(&stream_azul));

    // Aloca os canais na GPU:
    CHECA_CUDA(cudaMalloc(&d_vermelho, tam_char));
    CHECA_CUDA(cudaMalloc(&d_verde, tam_char));
    CHECA_CUDA(cudaMalloc(&d_azul, tam_char));
    CHECA_CUDA(cudaMalloc(&d_vermelho_aux, tam_int));
    CHECA_CUDA(cudaMalloc(&d_verde_aux, tam_int));
    CHECA_CUDA(cudaMalloc(&d_azul_aux, tam_int));

    dim3 dimGrade(ceil((float) x / LADO_BLOCO), ceil((float) y / LADO_BLOCO), 1);
    dim3 dimBloco(LADO_BLOCO, LADO_BLOCO, 1);

    // borra_X:
    CHECA_CUDA(cudaMemcpyAsync(d_vermelho, vermelho, tam_char, cudaMemcpyHostToDevice, stream_vermelho));
    borra_X<<<dimGrade, dimBloco, 0, stream_vermelho>>>(d_vermelho, d_vermelho_aux, y, x);

    CHECA_CUDA(cudaMemcpyAsync(d_verde, verde, tam_char, cudaMemcpyHostToDevice, stream_verde));
    borra_X<<<dimGrade, dimBloco, 0, stream_verde>>>(d_verde, d_verde_aux, y, x);

    CHECA_CUDA(cudaMemcpyAsync(d_azul, azul, tam_char, cudaMemcpyHostToDevice, stream_azul));
    borra_X<<<dimGrade, dimBloco, 0, stream_azul>>>(d_azul, d_azul_aux, y, x);

    // borra_Y:
    borra_Y<<<dimGrade, dimBloco, 0, stream_vermelho>>>(d_vermelho_aux, d_vermelho, y, x);
    CHECA_CUDA(cudaMemcpyAsync(vermelho, d_vermelho, tam_char, cudaMemcpyDeviceToHost, stream_vermelho));

    borra_Y<<<dimGrade, dimBloco, 0, stream_verde>>>(d_verde_aux, d_verde, y, x);
    CHECA_CUDA(cudaMemcpyAsync(verde, d_verde, tam_char, cudaMemcpyDeviceToHost, stream_verde));

    borra_Y<<<dimGrade, dimBloco, 0, stream_azul>>>(d_azul_aux, d_azul, y, x);
    CHECA_CUDA(cudaMemcpyAsync(azul, d_azul, tam_char, cudaMemcpyDeviceToHost, stream_azul));

    CHECA_CUDA(cudaStreamDestroy(stream_vermelho));
    CHECA_CUDA(cudaStreamDestroy(stream_verde));
    CHECA_CUDA(cudaStreamDestroy(stream_azul));

    CHECA_CUDA(cudaFree(d_vermelho));
    CHECA_CUDA(cudaFree(d_verde));
    CHECA_CUDA(cudaFree(d_azul));
    CHECA_CUDA(cudaFree(d_vermelho_aux));
    CHECA_CUDA(cudaFree(d_verde_aux));
    CHECA_CUDA(cudaFree(d_azul_aux));
}

unsigned char *aloca_vet(int tam) {
    return (unsigned char *) malloc(tam * sizeof(unsigned char));
}

void canais_da_imagem(
        const PPMImagem *img,
        unsigned char **vermelho,
        unsigned char **verde,
        unsigned char **azul, int tam) {
    *vermelho = aloca_vet(tam);
    *verde = aloca_vet(tam);
    *azul = aloca_vet(tam);

    for (int i = 0; i < tam; i++) {
        (*vermelho)[i] = img->imagem[i].vermelho;
        (*verde)[i] = img->imagem[i].verde;
        (*azul)[i] = img->imagem[i].azul;
    }
}

void imagem_dos_canais(
        const unsigned char *vermelho,
        const unsigned char *verde,
        const unsigned char *azul,
        PPMImagem *img, int tam) {
    for (int i = 0; i < tam; i++) {
        img->imagem[i].vermelho = vermelho[i];
        img->imagem[i].verde = verde[i];
        img->imagem[i].azul = azul[i];
    }
}

int main() {
    unsigned char *vermelho, *verde, *azul;// canais da imagem de entrada
    const char nome_arquivo[] = "/content/drive/MyDrive/secomp/3.ppm";
    const char nome_arquivo_out[] = "/content/drive/MyDrive/secomp/3_out.ppm";

    // Lê a imagem de entrada:
    PPMImagem *img = lePPM(nome_arquivo, 0);
    PPMImagem *img_output = lePPM(nome_arquivo, 1);
    int tam = img->x * img->y;

    canais_da_imagem(img, &vermelho, &verde, &azul, tam);
    auto comeco = std::chrono::high_resolution_clock::now();

    borra_imagem(vermelho, verde, azul, img->x, img->y);
    CHECA_CUDA(cudaDeviceSynchronize());// espera até os kerneis finalizarem

    auto fim = std::chrono::high_resolution_clock::now();
    imagem_dos_canais(vermelho, verde, azul, img_output, tam);

    escrevePPM(img_output, nome_arquivo_out);
    double tempo_gasto = duration_cast<milliseconds>(fim - comeco).count();
    printf("Tempo gasto: %.2lfms\n", tempo_gasto);

    // Libera os dados alocados:
    free(img->imagem);
    free(img);
    free(img_output->imagem);
    free(img_output);
    free(vermelho);
    free(verde);
    free(azul);

    return 0;
}

Tempo gasto: 125.00ms

